<a href="https://colab.research.google.com/github/AlexLeop/Minerando-Texto-do-Tweeter/blob/main/requisi%C3%A7%C3%A3o_de_palavras_chaves_no_Tweeter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **CONECTANDO E EXTRAINDO DADOS DO TWITTER**

In [ ]:
#bibliotecas necessárias
import json
from tweepy import OAuthHandler, Stream, StreamListener
from datetime import datetime

**CADASTRANDO CHAVES DE ACESSO**

In [ ]:
consumer_key = "HspCIea2zSlJG9GChiPBPhv3V"
consumer_secret = "OVXFgJkWbCweMR1rrLXFyE8KPQR6omqgRf16HF83SmBi5OVpCg"

access_token = "1441119614956539909-jI6FUmqU6qngVeuZiNFUHzee5wL7Ge"
access_token_secret = "YM0Gd25Y3MMeok2q7wGinXBpGDHUqzDCCMLaMAJ6GtK7i"

**DEFININDO ARQUIVO DE SAÍDA PARA ARMAZENAR TWEETS COLETADOS**

In [ ]:
data_hoje = datetime.now().strftime("%y-%m-%d-%H-%M-%S")
out = open(f"collected_tweets_{data_hoje}.txt", "w")

**IMPLEMENTANDO UMA CLASSE PARA CONECÇÃO COM O TWITTER**

In [ ]:
class MyListener(StreamListener):
    def on_data(self, data):
        itemString = json.dumps(data)
        out.write(itemString + " \n")
        return True
    def on_error(self, status):
        print(status)

**IMPLEMENTANDO FUNÇÃO MAIN**

In [ ]:
if __name__== "__main__":
    l = MyListener()
    auth = OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    
    stream = Stream(auth,l)
    stream.filter(track=["Trump"])

**REALIZANDO A LEITURA DA ESTRUTURA DE DADOS JASON**

In [ ]:
#Abrindo o arquivo dos Tweets e lendo as linhas
with open("/content/collected_tweets_21-11-24-02-56-48.txt", "r") as file:
    tweets = file.readlines()

**FAZENDO PARSE DO TWITTER PARA UM FORMATO JSON E EXPORTANDO ARQUIVO**

In [ ]:
#Testando e visualizando o parse da primeira linha de dados do twitter em formato JSON
with open("tweets.json", "w", encoding="utf-8") as f:

    json.dump(json.loads(json.loads(tweets[0])),f)
 

Extecutando o parse em todos os dados do Twitter de uma unica vez

In [ ]:
parse_tweets = [json.loads(json.loads(i)) for i in tweets]

QUANTIDADES DE TWEETS COLETADOS

In [ ]:
len(parse_tweets)

912

# **LIMPEZ E ORGANIZAÇÃO DOS DADOS DO TWITTER**

**TRANSFORMANDO OS DADOS DO TWITTER EM UM FORMATO RELACIONAL PARA SUBMETER À ANALISES**

In [ ]:
#Importando bibliotecas necessárias
import pandas as pd

In [ ]:
def tweet_para_df(tweet):
    try:
        df_tratado = pd.DataFrame(tweet).reset_index(drop = True).iloc[:1]
        df_tratado.drop(columns = ['quote_count','reply_count','retweet_count','favorite_count','favorited','retweeted','user','entities','retweeted_status'], inplace = True)

        df_tratado['user_id']               = tweet['user']['id']
        df_tratado['user_id_str']           = tweet['user']['id_str']
        df_tratado['user_screen_name']      = tweet['user']['screen_name']
        df_tratado['user_location']         = tweet['user']['location']
        df_tratado['user_description']      = tweet['user']['description']
        df_tratado['user_protected']        = tweet['user']['protected']
        df_tratado['user_verified']         = tweet['user']['verified']
        df_tratado['user_followers_count']  = tweet['user']['followers_count']
        df_tratado['user_friends_count']    = tweet['user']['friends_count']
        df_tratado['user_created_at']       = tweet['user']['created_at']

        user_mentions = []
        
        for i in range(len(tweet['entities']['user_mentions'])):
            dicionariobase = tweet['entities']['user_mentions'][i].copy()
            dicionariobase.pop('indices',None)
            df = pd.DataFrame(dicionariobase, index=[0])
            df = df.rename(columns = {
                'screen_name'   : 'entities_screen_name',
                'name'          : 'entities_name',
                'id'            : 'entities_id',
                'id_str'        : 'entities_id_str'
            })
            user_mentions.append(df)
        dfs = []
        for i in user_mentions:
            dfs.append(pd.concat([df_tratado.copy(), i ], axis=1))
        df_final = pd.concat(dfs, ignore_index = True)
    except:
        return None
    return df_final

In [ ]:
tweets.index(parse_tweets[0])